In [1]:
# from sklearn import svm, datasets 
# import sklearn.model_selection as model_selection
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import f1_score
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import ShuffleSplit,cross_val_score
# from sklearn import metrics


# #dataset
# #df = pd.read_csv(r'G:\MScProject\data\raw_col_norm.csv')
# df = pd.read_csv(r'G:\MScProject\data\svm_norm.csv')

# #choose features
# #dont need to filter out the first 30 rows, I have done this before
# X = df.iloc[:-30,2:]
# #X = df.iloc[:,2:35].drop(columns = ['rcap_hodl_waves','pi_cycle_top','ssr'])
# #y = df['up_do_1d']
# y = df.iloc[:-30,1]

# #split data
# n=X.shape[0]
# #cross validation split(shuffle split)
# crv = ShuffleSplit(n_splits = 5,test_size=0.2,random_state = 42)
# #X_train,X_test,y_train,y_test = model_selection.train_test_split(X,y,train_size = 0.75,test_size=0.25, random_state=101)


# #kernal methods

# rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1)
# poly = svm.SVC(kernel='poly', degree=3, C=1)

# poly_score = cross_val_score(poly,X,y,cv = crv)
# rbf_score = cross_val_score(rbf,X,y,cv = crv)
# poly_score

# # poly_pred = poly.predict(X_test)
# # rbf_pred = rbf.predict(X_test)


# # poly_accuracy = accuracy_score(y_test, poly_pred)
# # poly_f1 = f1_score(y_test, poly_pred, average='weighted')
# # print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
# # print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

# # rbf_accuracy = accuracy_score(y_test, rbf_pred)
# # rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
# # print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
# # print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

In [3]:
# by using built-in package
from sklearn import svm, datasets 
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit,cross_val_score,cross_validate
from sklearn import metrics
import statistics as sta
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from time import time




#dataset
#df = pd.read_csv(r'G:\MScProject\data\raw_col_norm.csv')
df = pd.read_csv(r'G:\MScProject\data\svm_norm.csv')

#choose features
#dont need to filter out the first 30 rows, I have done this before
X = df.iloc[:,2:]
y = df.iloc[:,1]
# X = df.iloc[:-30,2:]
# y = df.iloc[:-30,1]

#split data
n=X.shape[0]
#cross validation split(shuffle split)
crv = ShuffleSplit(n_splits = 10,test_size=0.1,random_state = 42)
#X_train,X_test,y_train,y_test = model_selection.train_test_split(X,y,train_size = 0.75,test_size=0.25, random_state=101)


para_gamma = [0.01,0.05,0.1,0.25,0.5,0.7,1.0,2.0,5.0,10]
para_d = [1,2,3,4,5,6,7]
para_C = [0.1,0.5,1,5,10,50,100]

# scoring = {'accuracy': 'precision','recall'}
score_list = ['accuracy','f1']


#-----------------------------------------------------------------------------------#
def para_tune(para_C,para_d_g,k,cv):
    # score_list = ['accuracy','f1']
    scoring = ['accuracy', 'f1_weighted']
    result=[]
    for d in para_d_g:
        for c in para_C:
            start = time()
            if k == 'poly':
                clf = svm.SVC(kernel='poly', degree=d, C=c,random_state = 42,decision_function_shape='ovo',class_weight={-1:0.17,0:0.372,1:0.455})
            elif k == 'rbf':
                clf = svm.SVC(kernel='rbf', gamma=d, C=c,random_state = 42,decision_function_shape='ovo',class_weight={-1:0.17,0:0.372,1:0.455})
            
            #--------------------------------------------------------------------------
            scores = cross_validate (clf, X, y, cv = cv, scoring = scoring,n_jobs = -1)
            # f1 =   cross_val_score(clf, X, y, cv = cv, scoring='f1_weighted')
            # a = sta.mean(accu)
            # f = sta.mean(f1)
            fit = np.mean(scores['fit_time'])
            a = sta.mean(scores['test_accuracy'])
            f = sta.mean(scores['test_f1_weighted'])
            end = time()
            time_cost = end-start
            # result.append({'d':d,'c':c,'avg_accu':a})
            result.append({'d':d,'c':c,'fit_time':fit,'avg_accu':a,'f1':f,'time_cost':time_cost})
    
    r = pd.DataFrame(result)

    return r


poly_res = para_tune(para_C,para_d,'poly',crv)
poly_res.sort_values(by=['avg_accu'],ascending=False)

# #-----------------------------------------------------------------------------------#
rbf_res = para_tune(para_gamma,para_C,'rbf',crv)
rbf_res.sort_values(by=['avg_accu'],ascending=False)



#get best parameters for rbf
# para_gamma = [0.01,0.05,0.1,0.25,0.5,0.7,1.0.,2.0]
# para_C = [0.1,0.5,1,5,10,50,100]
# kernel=['rbf', 'poly', 'sigmoid']
# param_grid = {'C': para_gamma, 'gamma': para_C,'kernel': kernel]}

# grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
# grid.fit(X_train,y_train)

,d,c,fit_time,avg_accu,f1,time_cost
19,0.5,10.0,0.108808,0.522124,0.454800,0.246339
29,1.0,10.0,0.103822,0.514159,0.450564,0.234439
9,0.1,10.0,0.110803,0.506195,0.431712,0.248336
28,1.0,5.0,0.103721,0.505310,0.437586,0.236368
18,0.5,5.0,0.110504,0.498230,0.425756,0.244346
...,...,...,...,...,...,...
42,10.0,0.1,0.111602,0.429204,0.259876,0.255317
56,50.0,1.0,0.128853,0.427434,0.259343,0.293216
44,10.0,0.5,0.119579,0.427434,0.259127,0.269280
59,50.0,10.0,0.181115,0.423894,0.396833,0.364027


In [4]:
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
20,3,100.0,0.458770,0.589381,0.551849,0.778919
13,2,100.0,0.218115,0.588496,0.521415,0.400928
19,3,50.0,0.332310,0.587611,0.538384,0.559502
12,2,50.0,0.178123,0.584071,0.516736,0.330117
18,3,10.0,0.151095,0.581416,0.514261,0.298202
29,5,0.5,0.150895,0.575221,0.510609,0.288230
25,4,10.0,0.256712,0.572566,0.527094,0.466750
24,4,5.0,0.200263,0.571681,0.513823,0.365022
17,3,5.0,0.142917,0.568142,0.502075,0.299201
30,5,1.0,0.178123,0.568142,0.510319,0.336101


In [5]:
poly_res = para_tune(para_C,para_d,'poly',10)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
2,1,1.0,0.083975,0.451525,0.327540,0.182511
1,1,0.5,0.110705,0.439112,0.269093,0.216420
0,1,0.1,0.101130,0.438219,0.267053,0.206446
3,1,5.0,0.103323,0.433676,0.335359,0.207445
4,1,10.0,0.121275,0.432830,0.337165,0.245345
6,1,100.0,0.224997,0.432032,0.344563,0.400926
7,2,0.1,0.086169,0.429283,0.277115,0.181515
5,1,50.0,0.180916,0.426691,0.336681,0.329121
8,2,0.5,0.097944,0.411457,0.299603,0.200970
9,2,1.0,0.103623,0.410619,0.313427,0.206450


In [6]:
poly_res = para_tune(para_C,para_d,'poly',15)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
6,1,100.0,0.229784,0.441778,0.356000,0.595409
5,1,50.0,0.176925,0.440000,0.351293,0.399929
0,1,0.1,0.095014,0.438222,0.267064,0.236370
1,1,0.5,0.087365,0.436444,0.273185,0.209438
4,1,10.0,0.121607,0.432889,0.325999,0.281248
7,2,0.1,0.092951,0.430222,0.279648,0.224398
3,1,5.0,0.105517,0.429333,0.314764,0.247338
2,1,1.0,0.091222,0.424889,0.286986,0.222405
9,2,1.0,0.105784,0.410667,0.306596,0.249332
15,3,0.5,0.107113,0.406222,0.311482,0.250330


In [7]:
poly_res = para_tune(para_C,para_d,'poly',20)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
6,1,100.0,0.264691,0.453650,0.372345,0.837760
4,1,10.0,0.125763,0.440006,0.341533,0.402922
0,1,0.1,0.098087,0.438221,0.267088,0.316152
5,1,50.0,0.208192,0.437594,0.350579,0.616351
1,1,0.5,0.090358,0.437328,0.269727,0.302192
7,2,0.1,0.107362,0.434649,0.275138,0.367018
2,1,1.0,0.094047,0.431971,0.281410,0.320143
3,1,5.0,0.112050,0.430122,0.323336,0.353055
8,2,0.5,0.115590,0.423042,0.313242,0.372006
9,2,1.0,0.127010,0.423011,0.325997,0.393945


In [8]:
poly_res = para_tune(para_C,para_d,'poly',25)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
4,1,10.0,0.147508,0.462222,0.358316,0.577963
6,1,100.0,0.346977,0.455111,0.375276,1.278155
5,1,50.0,0.268326,0.449778,0.367171,0.965921
3,1,5.0,0.165118,0.446222,0.332216,0.630314
0,1,0.1,0.106675,0.438222,0.267118,0.423869
7,2,0.1,0.110983,0.438222,0.274605,0.443812
1,1,0.5,0.151834,0.436444,0.268448,0.580446
2,1,1.0,0.109029,0.431111,0.274965,0.432860
10,2,5.0,0.183327,0.428444,0.358967,0.684469
9,2,1.0,0.132328,0.424889,0.330211,0.512139


In [9]:
poly_res = para_tune(para_C,para_d,'poly',30)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
6,1,100.0,0.380306,0.468160,0.385317,1.644342
4,1,10.0,0.202425,0.460503,0.357136,0.883636
3,1,5.0,0.181884,0.455927,0.346110,0.799574
5,1,50.0,0.272830,0.450379,0.365892,1.144783
0,1,0.1,0.117405,0.438146,0.267020,0.530093
1,1,0.5,0.118486,0.436392,0.267331,0.531106
7,2,0.1,0.158584,0.434637,0.269000,0.678329
2,1,1.0,0.119578,0.427691,0.275147,0.546322
8,2,0.5,0.195356,0.424158,0.310226,0.866783
9,2,1.0,0.173187,0.420626,0.325890,0.749956


In [44]:
# poly_res = para_tune(para_C,para_d,'poly',40)
# poly_res.sort_values(by=['f1'],ascending=False)

In [45]:
# poly_res = para_tune(para_C,para_d,'poly',50)
# poly_res.sort_values(by=['f1'],ascending=False)

In [46]:
# poly_res = para_tune(para_C,para_d,'poly',60)
# poly_res.sort_values(by=['f1'],ascending=False)

In [10]:
crv = ShuffleSplit(n_splits = 10,test_size=0.01,random_state = 42)
poly_res = para_tune(para_C,para_d,'poly',crv)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
18,3,10.0,0.207357,0.675000,0.618950,0.385504
13,2,100.0,0.280671,0.675000,0.624061,0.504708
12,2,50.0,0.253856,0.666667,0.614747,0.500209
17,3,5.0,0.178856,0.666667,0.613265,0.328766
29,5,0.5,0.198867,0.658333,0.605066,0.360035
23,4,1.0,0.174333,0.650000,0.596374,0.320142
35,6,0.1,0.189799,0.633333,0.581409,0.341595
20,3,100.0,0.626214,0.633333,0.597261,1.108027
11,2,10.0,0.165256,0.633333,0.582098,0.307652
19,3,50.0,0.501987,0.625000,0.593183,0.806033


In [11]:
crv = ShuffleSplit(n_splits = 10,test_size=0.02,random_state = 42)
poly_res = para_tune(para_C,para_d,'poly',crv)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
13,2,100.0,0.367716,0.621739,0.571015,0.632308
29,5,0.5,0.194080,0.617391,0.568682,0.366285
19,3,50.0,0.491485,0.608696,0.567169,0.847736
18,3,10.0,0.217916,0.608696,0.556316,0.392948
12,2,50.0,0.224943,0.608696,0.556436,0.403155
35,6,0.1,0.184406,0.600000,0.551630,0.337096
17,3,5.0,0.200563,0.600000,0.548063,0.425862
20,3,100.0,0.662413,0.595652,0.564235,1.102978
26,4,50.0,1.043882,0.591304,0.573957,1.794918
11,2,10.0,0.180625,0.586957,0.537440,0.351567


In [12]:
crv = ShuffleSplit(n_splits = 10,test_size=0.05,random_state = 42)
poly_res = para_tune(para_C,para_d,'poly',crv)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
13,2,100.0,0.304541,0.603509,0.541444,0.523114
19,3,50.0,0.428678,0.596491,0.551733,0.751531
12,2,50.0,0.309543,0.596491,0.532806,0.574749
18,3,10.0,0.199367,0.592982,0.531679,0.358042
27,4,100.0,1.605839,0.591228,0.574415,2.591434
20,3,100.0,0.685913,0.587719,0.551956,1.217149
11,2,10.0,0.176689,0.587719,0.520858,0.319163
17,3,5.0,0.295509,0.585965,0.523409,0.510635
26,4,50.0,1.020468,0.585965,0.562589,1.720798
23,4,1.0,0.175447,0.584211,0.521640,0.320177


In [13]:
crv = ShuffleSplit(n_splits = 10,test_size=0.1,random_state = 42)
poly_res = para_tune(para_C,para_d,'poly',crv)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
20,3,100.0,0.582214,0.589381,0.551849,0.946961
13,2,100.0,0.239459,0.588496,0.521415,0.434838
19,3,50.0,0.344488,0.587611,0.538384,0.607492
12,2,50.0,0.253928,0.584071,0.516736,0.484272
18,3,10.0,0.197261,0.581416,0.514261,0.367033
29,5,0.5,0.159971,0.575221,0.510609,0.300198
25,4,10.0,0.316852,0.572566,0.527094,0.557508
24,4,5.0,0.305482,0.571681,0.513823,0.544542
17,3,5.0,0.159087,0.568142,0.502075,0.356580
30,5,1.0,0.202957,0.568142,0.510319,0.380982


In [14]:
crv = ShuffleSplit(n_splits = 10,test_size=0.15,random_state = 42)
poly_res = para_tune(para_C,para_d,'poly',crv)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
20,3,100.0,0.521716,0.598817,0.565740,0.963925
19,3,50.0,0.371906,0.597633,0.552354,0.721074
13,2,100.0,0.303497,0.596450,0.532591,0.540080
18,3,10.0,0.228498,0.590533,0.524855,0.456598
12,2,50.0,0.164754,0.587574,0.520978,0.315637
25,4,10.0,0.358727,0.586391,0.542695,0.610118
26,4,50.0,0.800456,0.580473,0.561587,1.399108
24,4,5.0,0.220310,0.579290,0.521543,0.424866
30,5,1.0,0.274565,0.573964,0.517557,0.517613
32,5,10.0,0.882734,0.570414,0.550775,1.490250


In [15]:
crv = ShuffleSplit(n_splits = 10,test_size=0.2,random_state = 42)
poly_res = para_tune(para_C,para_d,'poly',crv)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
19,3,50.0,0.268178,0.590222,0.549509,0.480764
13,2,100.0,0.201061,0.589333,0.527988,0.375992
20,3,100.0,0.515205,0.584889,0.554064,0.839595
25,4,10.0,0.356488,0.583111,0.546283,0.598111
24,4,5.0,0.204613,0.579111,0.528704,0.399215
18,3,10.0,0.148304,0.579111,0.517096,0.282254
12,2,50.0,0.208314,0.576889,0.514408,0.426100
26,4,50.0,0.794294,0.574667,0.558028,1.286162
30,5,1.0,0.193567,0.572889,0.521813,0.405970
31,5,5.0,0.338791,0.569778,0.545546,0.645271


In [16]:
crv = ShuffleSplit(n_splits = 10,test_size=0.25,random_state = 42)
poly_res = para_tune(para_C,para_d,'poly',crv)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
13,2,100.0,0.162963,0.585461,0.527981,0.321140
19,3,50.0,0.254518,0.584397,0.545710,0.465765
12,2,50.0,0.201871,0.582624,0.522405,0.409429
20,3,100.0,0.468546,0.581915,0.554824,0.760952
18,3,10.0,0.172540,0.577305,0.517543,0.318668
25,4,10.0,0.208441,0.573759,0.538110,0.408907
24,4,5.0,0.155889,0.571986,0.523836,0.306692
31,5,5.0,0.308995,0.565248,0.546148,0.552140
17,3,5.0,0.116997,0.565248,0.504312,0.274288
30,5,1.0,0.201560,0.563475,0.516674,0.387961


In [17]:
crv = ShuffleSplit(n_splits = 10,test_size=0.3,random_state = 42)
poly_res = para_tune(para_C,para_d,'poly',crv)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
20,3,100.0,0.337502,0.578994,0.552411,0.577506
13,2,100.0,0.143615,0.573669,0.516812,0.295212
19,3,50.0,0.300097,0.573373,0.538514,0.504650
25,4,10.0,0.190875,0.571302,0.536364,0.368820
12,2,50.0,0.143613,0.568935,0.508885,0.283241
18,3,10.0,0.108409,0.567456,0.507556,0.229385
26,4,50.0,0.501844,0.563018,0.550861,0.915107
24,4,5.0,0.172648,0.560947,0.513834,0.374054
30,5,1.0,0.125862,0.556805,0.507566,0.255317
17,3,5.0,0.083681,0.555917,0.494096,0.195493


In [18]:
crv = ShuffleSplit(n_splits = 10,test_size=0.4,random_state = 42)
poly_res = para_tune(para_C,para_d,'poly',crv)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1,time_cost
13,2,100.0,0.107642,0.578889,0.524879,0.215001
19,3,50.0,0.209438,0.573333,0.541207,0.375993
20,3,100.0,0.294412,0.571333,0.548702,0.583439
12,2,50.0,0.084274,0.569556,0.510611,0.191710
25,4,10.0,0.167252,0.568889,0.537056,0.401924
24,4,5.0,0.139327,0.563111,0.520512,0.267284
18,3,10.0,0.118067,0.561778,0.505638,0.295541
30,5,1.0,0.089759,0.556889,0.511762,0.197471
17,3,5.0,0.067859,0.552889,0.493683,0.168235
29,5,0.5,0.073010,0.548000,0.494059,0.169056


In [56]:
crv = ShuffleSplit(n_splits = 10,test_size=0.5,random_state = 42)
poly_res = para_tune(para_C,para_d,'poly',crv)
poly_res.sort_values(by=['avg_accu'],ascending=False)

,d,c,fit_time,avg_accu,f1
13,2,100.0,0.010671,0.816944,0.816630
20,3,100.0,0.010072,0.807309,0.815898
19,3,50.0,0.010072,0.806312,0.812775
6,1,100.0,0.007480,0.800997,0.743811
12,2,50.0,0.008477,0.796678,0.783444
27,4,100.0,0.009973,0.795681,0.804228
26,4,50.0,0.009973,0.794020,0.802848
5,1,50.0,0.006482,0.787043,0.715373
25,4,10.0,0.009475,0.786047,0.790832
11,2,10.0,0.007081,0.779070,0.717325


In [57]:
# crv = ShuffleSplit(n_splits = 10,test_size=0.6,random_state = 42)
# poly_res = para_tune(para_C,para_d,'poly',crv)
# poly_res.sort_values(by=['f1'],ascending=False)

In [58]:
# poly_res.sort_values(by=['avg_accu'],ascending=False)
# y.hist()
# # rbf_res = para_tune(para_gamma,para_C,'rbf')
# # rbf_res.sort_values(by=['avg_accu'],ascending=False)
# poly_res.sort_values(by=['avg_accu'],ascending=False)


